## Content Based Filtering
영화의 콘텐츠(overview, cast, crew, keyword, tagline etc) 를 사용하여 다른 영화와 유사성을 찾고, 유사할 가능성이 가장 높은 영화를 추천

### Plot description based Recommender
모든 영화-영화 쌍의 유사성 점수를 계산하고 이를 기반으로 영화 추천

In [1]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_csv('../input/tmdb-movie-metadata/tmdb_5000_credits.csv')
df2 = pd.read_csv('../input/tmdb-movie-metadata/tmdb_5000_movies.csv')

In [3]:
df2['overview'].head(5)

0    In the 22nd century, a paraplegic Marine is di...
1    Captain Barbossa, long believed to be dead, ha...
2    A cryptic message from Bond’s past sends him o...
3    Following the death of District Attorney Harve...
4    John Carter is a war-weary, former military ca...
Name: overview, dtype: object

### compute Term Frequency-Inverse Document Frequency (TF-IDF) vectors for each overview
- TF(term frequency) = term instances/total instances
- IDF(Inverse Document Frequency) = number of documents/documents with term
- overall importance of each word = TF * IDF
  - 각 열은 a word in the overview vocabulary 을 나타내고
  - 각 행은 영화를 나타냄
This is done to reduce the importance of words that occur frequently in plot overviews and therefore, their significance in computing the final similarity score.